# hicstuff API demo

All steps of hicstuff pipeline and downstream operations can be ran using the python api. The matrix can be generated directly from the reads, just like in the command line or by doing each step separately for fine control.


## Data preparation
If using minimap2, the genome can be in fasta format. If using bowtie2, it must be indexed using bowtie2-build:
```bash
bowtie2-build genome.fasta genome
```
The input reads can be in fastq format, or if already aligned to the genome in SAM/BAM format. The pipeline also accepts input in the form of pairs file. The input format is specified using the `start_stage` argument (fastq/sam/pairs/pairs_idx).

## Full pipeline using the API
The hicstuff.pipeline submodule allows to run all steps at one, in a way identical to the hicstuff pipeline command.

In [7]:
import hicstuff.pipeline as hpi

hpi.full_pipeline(input1='../../test_data/sample.reads_for.fastq.gz', 
                  input2='../../test_data/sample.reads_rev.fastq.gz', 
                  genome='../../test_data/genome/seq', 
                  enzyme="DpnII",
                  filter_events=True, 
                  pcr_duplicates=True)

INFO :: ## hicstuff: v1.2.16 log file
INFO :: ## date: 2019-04-24 01:42:50
INFO :: ## enzyme: DpnII
INFO :: ## input1: ../../test_data/sample.reads_for.fastq.gz 
INFO :: ## input2: ../../test_data/sample.reads_rev.fastq.gz
INFO :: ## ref: ../../test_data/genome/seq
INFO :: ---
INFO :: Genome digested into 564 fragments with a median length of 87.5
INFO :: 87% reads (single ends) mapped with Q >= 30 (17442/20000)
INFO :: Filtering with thresholds: uncuts=7 loops=9
INFO :: Proportion of inter contacts: 4.51% (intra: 233, inter: 11)
INFO :: 7356 pairs discarded: Loops: 21, Uncuts: 7335, Weirds: 0
INFO :: 244 pairs kept (3.21%)
INFO :: 0% PCR duplicates have been filtered out (0 / 244 pairs) 
INFO :: 244 pairs used to build a contact map of 564 bins with 245 nonzero entries.
INFO :: Contact map generated after 0h 0m 1s
